# Connection to blob

In [0]:
storageAccount = "datalakefabdeprojetos"
containerName = "xxxxxxx"
mountpoint = "/mnt/xxxxxx/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope="xxxxxx",key="xxxxx")
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"
try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails


/mnt/ingrid-sollim/ has been mounted


In [0]:
path=mountpoint+"bikes"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
dbfs:/mnt/ingrid-sollim/bikes/customer.csv,customer.csv,1813963,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/person.csv,person.csv,13646947,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/product.csv,product.csv,104823,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderdetail.csv,salesorderdetail.csv,13801182,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderheader.csv,salesorderheader.csv,8267704,1694474862000
dbfs:/mnt/ingrid-sollim/bikes/spoffer.csv,spoffer.csv,36680,1694474860000


In [0]:
df = spark.read.option("header",True).options(delimiter=";").csv(path+"/product.csv")
display(df.limit(10))

ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,Adjustable Race,AR-5381,0,0,NULL,1000,750,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
2,Bearing Ball,BA-8327,0,0,NULL,1000,750,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
3,BB Ball Bearing,BE-2349,1,0,NULL,800,600,"0,00","0,00",NULL,NULL,NULL,NULL,1,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
4,Headset Ball Bearings,BE-2908,0,0,NULL,800,600,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
316,Blade,BL-2036,1,0,NULL,800,600,"0,00","0,00",NULL,NULL,NULL,NULL,1,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827
317,LL Crankarm,CA-5965,0,0,Black,500,375,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,L,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827
318,ML Crankarm,CA-6738,0,0,Black,500,375,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,M,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,EABB9A92-FA07-4EAB-8955-F0517B4A4CA7,2014-02-08 10:01:36.827
319,HL Crankarm,CA-7457,0,0,Black,500,375,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,7D3FD384-4F29-484B-86FA-4206E276FE58,2014-02-08 10:01:36.827
320,Chainring Bolts,CB-2903,0,0,Silver,1000,750,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,7BE38E48-B7D6-4486-888E-F53C26735101,2014-02-08 10:01:36.827
321,Chainring Nut,CN-6137,0,0,Silver,1000,750,"0,00","0,00",NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00.000,NULL,NULL,3314B1D7-EF69-4431-B6DD-DC75268BD5DF,2014-02-08 10:01:36.827


In [0]:
from pyspark.sql.functions import expr, col, regexp_replace,regexp_extract,when, to_date,to_timestamp

#Check for missing data

In [0]:
#Replace NULL string for null values 
def replace_string_null(df, columns):
    for column_name in columns:
        df = df.withColumn(column_name, when(col(column_name) == "NULL", None).otherwise(col(column_name)))
    return df

# Example usage:
columns_to_replace = df.columns
null_replaced = replace_string_null(df, columns_to_replace)
    

display(null_replaced.limit(10))


ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,Adjustable Race,AR-5381,0,0,null,1000,750,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
2,Bearing Ball,BA-8327,0,0,null,1000,750,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
3,BB Ball Bearing,BE-2349,1,0,null,800,600,"0,00","0,00",null,null,null,null,1,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
4,Headset Ball Bearings,BE-2908,0,0,null,800,600,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
316,Blade,BL-2036,1,0,null,800,600,"0,00","0,00",null,null,null,null,1,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827
317,LL Crankarm,CA-5965,0,0,Black,500,375,"0,00","0,00",null,null,null,null,0,null,L,null,null,null,2008-04-30 00:00:00.000,null,null,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827
318,ML Crankarm,CA-6738,0,0,Black,500,375,"0,00","0,00",null,null,null,null,0,null,M,null,null,null,2008-04-30 00:00:00.000,null,null,EABB9A92-FA07-4EAB-8955-F0517B4A4CA7,2014-02-08 10:01:36.827
319,HL Crankarm,CA-7457,0,0,Black,500,375,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,7D3FD384-4F29-484B-86FA-4206E276FE58,2014-02-08 10:01:36.827
320,Chainring Bolts,CB-2903,0,0,Silver,1000,750,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,7BE38E48-B7D6-4486-888E-F53C26735101,2014-02-08 10:01:36.827
321,Chainring Nut,CN-6137,0,0,Silver,1000,750,"0,00","0,00",null,null,null,null,0,null,null,null,null,null,2008-04-30 00:00:00.000,null,null,3314B1D7-EF69-4431-B6DD-DC75268BD5DF,2014-02-08 10:01:36.827


In [0]:
display(null_replaced.select("ProductLine").distinct())

ProductLine
null
S
M
R
T


#Change Data types

In [0]:
null_replaced.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: string (nullable = true)
 |-- FinishedGoodsFlag: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: string (nullable = true)
 |-- ReorderPoint: string (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- DaysToManufacture: string (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: string (nullable = true)
 |-- ProductModelID: string (nullable = true)
 |-- SellStartDate: string (nullable = true)
 |-- SellEndDate: string (nullable = true)
 |-- DiscontinuedDate: string (nullable 

In [0]:
#replace comma for point
comma_replaced = null_replaced.withColumn("StandardCost",regexp_replace("StandardCost",",",".")).\
withColumn("ListPrice",regexp_replace("ListPrice",",","."))


In [0]:

product = comma_replaced.withColumn("ProductID",col("ProductID").cast("int"))\
.withColumn("ProductNumber",col("ProductNumber").cast("int"))\
.withColumn("FinishedGoodsFlag",col("FinishedGoodsFlag").cast("int"))\
.withColumn("SafetyStockLevel",col("SafetyStockLevel").cast("int"))\
.withColumn("ReorderPoint",col("ReorderPoint").cast("int"))\
.withColumn("StandardCost",col("StandardCost").cast("float"))\
.withColumn("ListPrice",col("ListPrice").cast("float"))\
.withColumn("Weight",col("Weight").cast("float"))\
.withColumn("DaysToManufacture",col("DaysToManufacture").cast("int"))\
.withColumn("SellStartDate",to_date(to_timestamp(col("SellStartDate"), "yyyy-MM-dd HH:mm:ss.SSS"),"yyyy-MM-dd"))\
.withColumn("SellEndDate",to_date(to_timestamp(col("SellEndDate"), "yyyy-MM-dd HH:mm:ss.SSS"),"yyyy-MM-dd"))\
.withColumn("ModifiedDate",to_timestamp(col("ModifiedDate"),"yyyy-MM-dd HH:mm:ss.SSS"))

display(product.limit(10))

ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,Adjustable Race,null,0,0,null,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08T10:01:36.827+0000
2,Bearing Ball,null,0,0,null,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08T10:01:36.827+0000
3,BB Ball Bearing,null,1,0,null,800,600,0.0,0.0,null,null,null,null,1,null,null,null,null,null,2008-04-30,null,null,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08T10:01:36.827+0000
4,Headset Ball Bearings,null,0,0,null,800,600,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08T10:01:36.827+0000
316,Blade,null,1,0,null,800,600,0.0,0.0,null,null,null,null,1,null,null,null,null,null,2008-04-30,null,null,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08T10:01:36.827+0000
317,LL Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,L,null,null,null,2008-04-30,null,null,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08T10:01:36.827+0000
318,ML Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,M,null,null,null,2008-04-30,null,null,EABB9A92-FA07-4EAB-8955-F0517B4A4CA7,2014-02-08T10:01:36.827+0000
319,HL Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,7D3FD384-4F29-484B-86FA-4206E276FE58,2014-02-08T10:01:36.827+0000
320,Chainring Bolts,null,0,0,Silver,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,7BE38E48-B7D6-4486-888E-F53C26735101,2014-02-08T10:01:36.827+0000
321,Chainring Nut,null,0,0,Silver,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,3314B1D7-EF69-4431-B6DD-DC75268BD5DF,2014-02-08T10:01:36.827+0000


In [0]:
product.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: integer (nullable = true)
 |-- MakeFlag: string (nullable = true)
 |-- FinishedGoodsFlag: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- StandardCost: float (nullable = true)
 |-- ListPrice: float (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: float (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: string (nullable = true)
 |-- ProductModelID: string (nullable = true)
 |-- SellStartDate: date (nullable = true)
 |-- SellEndDate: date (nullable = true)
 |-- DiscontinuedDate: string (nullable =

In [0]:
product.columns

Out[19]: ['ProductID',
 'Name',
 'ProductNumber',
 'MakeFlag',
 'FinishedGoodsFlag',
 'Color',
 'SafetyStockLevel',
 'ReorderPoint',
 'StandardCost',
 'ListPrice',
 'Size',
 'SizeUnitMeasureCode',
 'WeightUnitMeasureCode',
 'Weight',
 'DaysToManufacture',
 'ProductLine',
 'Class',
 'Style',
 'ProductSubcategoryID',
 'ProductModelID',
 'SellStartDate',
 'SellEndDate',
 'DiscontinuedDate',
 'rowguid',
 'ModifiedDate']

In [0]:
column_mapping = {
'ProductID':'product_id',
 'Name':'name',
 'ProductNumber':'product_number',
 'MakeFlag':'make_flag',
 'FinishedGoodsFlag':'finishedGoods_flag',
 'Color':'color',
 'SafetyStockLevel':'safety_stock_level',
 'ReorderPoint':'reorder_point',
 'StandardCost':'standard_cost',
 'ListPrice':'list_price',
 'Size':'size',
 'SizeUnitMeasureCode':'size_unit_measure_code',
 'WeightUnitMeasureCode':'weight_unit_measure_code',
 'Weight':'weight',
 'DaysToManufacture':'days_to_manufacture',
 'ProductLine':'product_line',
 'Class':'class',
 'Style':'style',
 'ProductSubcategoryID':'product_subcategory_id',
 'ProductModelID':'product_model_id',
 'SellStartDate':'sell_start_date',
 'SellEndDate':'sell_end_date',
 'DiscontinuedDate':'discontinued_date',
 'rowguid':'rowguid',
 'ModifiedDate':'modified_date'
}

# Create a list of column expressions with aliases
select_expr = [col(old_col).alias(new_col) for old_col, new_col in column_mapping.items()]

# Apply the column renaming using select and alias
df_renamed = product.select(*select_expr)
display(df_renamed.limit(10))

product_id,name,product_number,make_flag,finishedGoods_flag,color,safety_stock_level,reorder_point,standard_cost,list_price,size,size_unit_measure_code,weight_unit_measure_code,weight,days_to_manufacture,product_line,class,style,product_subcategory_id,product_model_id,sell_start_date,sell_end_date,discontinued_date,rowguid,modified_date
1,Adjustable Race,null,0,0,null,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08T10:01:36.827+0000
2,Bearing Ball,null,0,0,null,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08T10:01:36.827+0000
3,BB Ball Bearing,null,1,0,null,800,600,0.0,0.0,null,null,null,null,1,null,null,null,null,null,2008-04-30,null,null,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08T10:01:36.827+0000
4,Headset Ball Bearings,null,0,0,null,800,600,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08T10:01:36.827+0000
316,Blade,null,1,0,null,800,600,0.0,0.0,null,null,null,null,1,null,null,null,null,null,2008-04-30,null,null,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08T10:01:36.827+0000
317,LL Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,L,null,null,null,2008-04-30,null,null,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08T10:01:36.827+0000
318,ML Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,M,null,null,null,2008-04-30,null,null,EABB9A92-FA07-4EAB-8955-F0517B4A4CA7,2014-02-08T10:01:36.827+0000
319,HL Crankarm,null,0,0,Black,500,375,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,7D3FD384-4F29-484B-86FA-4206E276FE58,2014-02-08T10:01:36.827+0000
320,Chainring Bolts,null,0,0,Silver,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,7BE38E48-B7D6-4486-888E-F53C26735101,2014-02-08T10:01:36.827+0000
321,Chainring Nut,null,0,0,Silver,1000,750,0.0,0.0,null,null,null,null,0,null,null,null,null,null,2008-04-30,null,null,3314B1D7-EF69-4431-B6DD-DC75268BD5DF,2014-02-08T10:01:36.827+0000


In [0]:
df_renamed.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- product_number: integer (nullable = true)
 |-- make_flag: string (nullable = true)
 |-- finishedGoods_flag: integer (nullable = true)
 |-- color: string (nullable = true)
 |-- safety_stock_level: integer (nullable = true)
 |-- reorder_point: integer (nullable = true)
 |-- standard_cost: float (nullable = true)
 |-- list_price: float (nullable = true)
 |-- size: string (nullable = true)
 |-- size_unit_measure_code: string (nullable = true)
 |-- weight_unit_measure_code: string (nullable = true)
 |-- weight: float (nullable = true)
 |-- days_to_manufacture: integer (nullable = true)
 |-- product_line: string (nullable = true)
 |-- class: string (nullable = true)
 |-- style: string (nullable = true)
 |-- product_subcategory_id: string (nullable = true)
 |-- product_model_id: string (nullable = true)
 |-- sell_start_date: date (nullable = true)
 |-- sell_end_date: date (nullable = true)
 |-- discontinu

#Ingest the cleaned data to the database

### Connection to the database

In [0]:
# Get the secrets from the jdbc secret scope.
username = dbutils.secrets.get(scope="xxxxx", key="xxxxx")
password = dbutils.secrets.get(scope="xxxxxx", key="xxxxx")

jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "xxxxxx"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.product'

### Insert into the database


In [0]:
df_renamed.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()

In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[20]: True